### ***`Imports`***

In [1]:
import tensorflow as tf
from keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

##### Hyperparameters

In [2]:
IMG_SIZE = 200
Batch_Size = 16
Learing_Rate = 0.0001
Epochs = 15

##### Data Generation

In [3]:
base_dir = 'IMG_CLASSES\Cancer'
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
    )
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_data = train_datagen.flow_from_directory(
    base_dir,
    shuffle=True,
    target_size=(IMG_SIZE, IMG_SIZE),  
    batch_size=Batch_Size,
    class_mode='binary', 
    subset='training',
    seed=42
)

In [ ]:
val_data = train_datagen.flow_from_directory(
    base_dir,
    shuffle=True,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=Batch_Size,
    class_mode='binary',
    subset='validation',
    seed=42
)

Found 2248 images belonging to 2 classes.


### ***`Model`***


In [6]:
base_model = EfficientNetB0(weights=None, include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=Learing_Rate), loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
# If you want to continue training the loaded model:
history = model.fit(
    train_data,
    steps_per_epoch=train_data.samples // train_data.batch_size,  # Adjust based on the number of images in your training set
    epochs=Epochs,
    validation_data=val_data,
    validation_steps=val_data.samples // val_data.batch_size)  # Adjust based on the number of images in your validation set
loss, accuracy = model.evaluate(val_data)

# Make predictions on the validation set
y_true = val_data.classes
y_pred_prob = model.predict(val_data)
y_pred = (y_pred_prob > 0.75).astype(int)

# Compute confusion matrix
confusion = confusion_matrix(y_true, y_pred)

print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")
print("Accuracy:", round(accuracy*100, 2), '%')
print("Confusion matrix:\n", confusion)

# Save the model
model.save(f'EfficientNet_Binary({accuracy * 100:.2f}%).h5')

Epoch 1/15
562/562 [==============================] - 97s 156ms/step - loss: 0.3867 - accuracy: 0.8363 - val_loss: 1.6209 - val_accuracy: 0.5746
Epoch 2/15
562/562 [==============================] - 86s 153ms/step - loss: 0.2249 - accuracy: 0.9120 - val_loss: 0.5285 - val_accuracy: 0.9205
Epoch 3/15
562/562 [==============================] - 86s 153ms/step - loss: 0.1754 - accuracy: 0.9356 - val_loss: 0.1398 - val_accuracy: 0.9598
Epoch 4/15
562/562 [==============================] - 86s 152ms/step - loss: 0.1355 - accuracy: 0.9510 - val_loss: 0.2072 - val_accuracy: 0.9558
Epoch 5/15
562/562 [==============================] - 86s 153ms/step - loss: 0.1119 - accuracy: 0.9595 - val_loss: 0.1000 - val_accuracy: 0.9710
Epoch 6/15
562/562 [==============================] - 86s 153ms/step - loss: 0.1057 - accuracy: 0.9624 - val_loss: 0.1245 - val_accuracy: 0.9594
Epoch 7/15
562/562 [==============================] - 86s 153ms/step - loss: 0.0753 - accuracy: 0.9746 - val_loss: 0.1398 - val_ac